In [11]:
import pandas as pd
import psycopg2
import requests

In [12]:
# Scripts de creacion de tablas

sql_especies = """ CREATE TABLE if not exists fmorosini_coderhouse.especies (
	id int NOT NULL,
	nombrevulgar varchar(50) NOT NULL,
	nombrecientifico varchar(50) NOT NULL,
	imagen varchar(50) NOT NULL,
	magnitud int4 NOT NULL,
	tipo varchar(15) NOT NULL,
	follaje varchar(15) NOT NULL,
	url_ficha varchar(255) NULL,
	thumbnail varchar(100) NULL,
	CONSTRAINT especies_pkey PRIMARY KEY (id)
); """


sql_localidades = """ CREATE TABLE if not exists fmorosini_coderhouse.localidades (
	id int NOT NULL,
	nombre varchar(80) NULL,
	CONSTRAINT localidades_pk PRIMARY KEY (id)
); """

sql_arboles = """ CREATE TABLE if not exists fmorosini_coderhouse.arboles (
	id int4 NOT NULL,
	lat float,
	lon float,
	especie int4 NOT NULL,
	localidad int4 NOT NULL,
	CONSTRAINT clave PRIMARY KEY (id)
); """

sql_FK_1 = """ ALTER TABLE fmorosini_coderhouse.arboles ADD CONSTRAINT "FK_especie" FOREIGN KEY (especie) REFERENCES fmorosini_coderhouse.especies(id); """
sql_FK_2 = """ ALTER TABLE fmorosini_coderhouse.arboles ADD CONSTRAINT "FK_localidad" FOREIGN KEY (localidad) REFERENCES fmorosini_coderhouse.localidades(id); """

In [13]:
# Defino connectionstring y abro conexión

con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

In [14]:
# Corro los sql de creación de tablas y Foreign Keys

cursor = con.cursor()

cursor.execute(sql_especies)
cursor.execute(sql_localidades)
cursor.execute(sql_arboles)
con.commit()

try:
    cursor.execute(sql_FK_1)
    cursor.execute(sql_FK_2)
    con.commit()
except:
    print("Error, las FK ya están creadas")
    con.rollback()

con.close()

Error, las FK ya están creadas


In [41]:
# URLs de endpoints de las 3 entidades

url_especies = "https://api.arbolesurbanos.com.ar/json/especies"
url_localidades = "https://api.arbolesurbanos.com.ar/json/localidades"
url_arboles = "https://api.arbolesurbanos.com.ar/json/arboles"

# Traigo la data

req_especies = requests.get(url_especies)
req_localidades = requests.get(url_localidades)
req_arboles = requests.get(url_arboles)

# Convierto a Dict

especies = req_especies.json()["data"]
localidades = req_localidades.json()["data"]
arboles = req_arboles.json()["data"]

# Cargo DataFrames con los dict

especies_df = pd.DataFrame.from_dict(especies)
localidades_df = pd.DataFrame.from_dict(localidades)

localidades_df.head(10)

,nombre,zoom,posicion,ogc_fid
0,San Martín de Los Andes,13,"{'type': 'Point', 'coordinates': [-71.27603530...",1
1,San Carlos de Bariloche,12,"{'type': 'Point', 'coordinates': [-71.31922576...",2
2,El Bolsón,12,"{'type': 'Point', 'coordinates': [-71.53703257...",3
3,Junín de Los Andes,12,"{'type': 'Point', 'coordinates': [-71.07591954...",4
4,Aluminé,12,"{'type': 'Point', 'coordinates': [-70.90988583...",5
5,Neuquén Capital,12,"{'type': 'Point', 'coordinates': [-68.05924099...",11
6,Zapala,12,"{'type': 'Point', 'coordinates': [-70.05838050...",14


In [42]:
especies_df.head(10)

,id,nombrecientifico,nombrevulgar,magnitud,tipo,follaje,imagen,thumbnail,url_ficha
0,144,Abies alba,Abeto blanco,1,Conífera,Perenne,ABIES ALBA.gif,/iconos/1/ABIES ALBA.gif,https://www.arbolesurbanos.com.ar/abies-alba-a...
1,160,Abies concolor,Abeto del Colorado,1,Conífera,Perenne,Abies concolor.gif,/thumbnails/abies-concolor.jpg,
2,111,Abies grandis,Abeto de Vancuver,1,Conífera,Perenne,Abies grandis.gif,/iconos/1/Abies grandis.gif,https://www.arbolesurbanos.com.ar/abies-grandi...
3,166,Abies procera,Abeto noble,1,Conífera,Perenne,Abies procera.gif,/thumbnails/abies-procera.jpg,https://www.arbolesurbanos.com.ar/abies-porcer...
4,112,Acacia dealbata,Aromo,3,Latifoliada,Perenne,Acacia dealbata.gif,/iconos/3/Acacia dealbata.gif,https://www.arbolesurbanos.com.ar/acacia-dealb...
5,206,Acacia melanoxylon,Acacia negra,2,Latifoliada,Perenne,,latifoliada_perenne.gif,None
6,113,Acer campestris,Arce campestre,3,Latifoliada,Caduco,Acer campestris.gif,/thumbnails/acer-campestris.jpg,https://www.arbolesurbanos.com.ar/acer-campest...
7,207,Acer ginnala,Acer tártaro,3,Latifoliada,Caduco,,latifoliada_perenne.gif,None
8,114,Acer negundo,Negundo,3,Latifoliada,Caduco,ACER NEGUNDO.gif,/thumbnails/acer-negundo.jpg,https://www.arbolesurbanos.com.ar/acer-negundo...
9,50,Acer palmatum,Arce japones,3,Latifoliada,Caduco,ACER JAPONICA.gif,/thumbnails/arce-japones.JPG,https://www.arbolesurbanos.com.ar/acer-palmatu...


In [17]:
arboles

[{'id': 5407,
  'especie': 12,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91609438689663, -39.238584250266065]}},
 {'id': 5393,
  'especie': 3,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91629903804633, -39.2388849493729]}},
 {'id': 5394,
  'especie': 3,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91558995069984, -39.238039441881824]}},
 {'id': 5361,
  'especie': 121,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91561055017057, -39.23838603411222]}},
 {'id': 5359,
  'especie': 121,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91617750233583, -39.23826005597549]}},
 {'id': 5360,
  'especie': 121,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91565441549842, -39.23851147883219]}},
 {'id': 5395,
  'especie': 3,
  'localidad': 5,
  'posicion': {'type': 'Point',
   'coordinates': [-70.91566324345436, -39.23803290804034]}}

In [43]:
# Al de arboles hay que laburarlo un poco más porque está en GeoJson, un Json específico para guardar datos georreferenciados

arboles_list = []

for arbol in arboles:

    arbol_dict = {}

    arbol_dict["id"] = arbol["id"]
    arbol_dict["localidad"] = arbol["localidad"]
    arbol_dict["especie"] = arbol["especie"]
    arbol_dict["lon"] = arbol["posicion"]["coordinates"][0]
    arbol_dict["lat"] = arbol["posicion"]["coordinates"][1]

    arboles_list.append(arbol_dict)

In [44]:
arboles_df = pd.DataFrame.from_dict(arboles_list)

arboles_df.head(10)

,id,localidad,especie,lon,lat
0,5407,5,12,-70.916094,-39.238584
1,5393,5,3,-70.916299,-39.238885
2,5394,5,3,-70.915590,-39.238039
3,5361,5,121,-70.915611,-39.238386
4,5359,5,121,-70.916178,-39.238260
5,5360,5,121,-70.915654,-39.238511
6,5395,5,3,-70.915663,-39.238033
7,5367,5,121,-70.915884,-39.238069
8,5401,5,57,-70.915812,-39.238866
9,5405,5,25,-70.915801,-39.238033


In [45]:
# Vuelvo a abrir conexión para eliminar datos de tablas

con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

cursor = con.cursor()

In [46]:
sql_delete_arboles = """ DELETE FROM arboles WHERE 1 = 1 """
sql_delete_especies  = """ DELETE FROM especies WHERE 1 = 1 """
sql_delete_localidades = """ DELETE FROM localidades WHERE 1 = 1 """

cursor.execute(sql_delete_arboles)
cursor.execute(sql_delete_especies)
cursor.execute(sql_delete_localidades)

con.commit()

con.close()

In [47]:
con = psycopg2.connect("dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port= 5439 user=fmorosini_coderhouse password= 77Vz3W9KPP")

def inserta_localidades(fila):

    cursor = con.cursor()

    cursor.execute(""" INSERT INTO localidades (id,nombre) VALUES (%s,%s)""",(fila["ogc_fid"],fila["nombre"]))

    con.commit()

def inserta_especies(fila):

    cursor = con.cursor()

    cursor.execute(""" INSERT INTO especies (id,nombrevulgar,nombrecientifico,imagen,magnitud,tipo,follaje,url_ficha,thumbnail) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(fila["id"],fila["nombrevulgar"],fila["nombrecientifico"],fila["imagen"],fila["magnitud"],fila["tipo"],fila["follaje"],fila["url_ficha"],fila["thumbnail"]))

    con.commit()



localidades_df.apply(inserta_localidades,axis=1)
especies_df.apply(inserta_especies,axis=1)

con.close()